#Algorithms in Structural Biology 

I. Emiris and E. Chrysina, Assignment 2

Announced: 08.04.2020   Deadline: 26.04.2020

Michael Batskinis (DS2190011)


Import packages

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix

B. Calculate the c-RMSD for the comparison of the structures in (a) using your software from Assignment 1, or any available tool, if you didn’t finish your own: Make sure you download the coordinates in .pdb format and NOT .cif format because this might not be readable by selected software. So you shall calculate c-RMSD over:
1.   all atoms
2.   over Ca atoms
3.   over main chain atoms





In [2]:
#Loading coordinates of the aligned atoms of the two protein structures
df_1uwc = pd.read_csv("1UWC_aligned.pdb", sep = "\s+",header = None, names = ['atom_type','x','y','z'], usecols=[2,6,7,8],skiprows=1)
df_6fat = pd.read_csv("6fat_aligned.pdb", sep = "\s+",header = None, names = ['atom_type','x','y','z'], usecols=[2,6,7,8],skiprows=1)

#ALL IDENTICAL ATOMS
common_1uwc = pd.read_csv("1uwc_identical.pdb", sep = "\s+",header = None, names = ['x','y','z'], usecols=[6,7,8],skiprows=1)
common_6fat = pd.read_csv("6fat_identical.pdb", sep = "\s+",header = None, names = ['x','y','z'], usecols=[6,7,8],skiprows=1)
#ONLY CA ATOMS
CA_6fat = df_6fat[df_6fat['atom_type']=='CA']
CA_6fat = CA_6fat.drop(['atom_type'],axis=1)

CA_1uwc = df_1uwc[df_1uwc['atom_type']=='CA']
CA_1uwc = CA_1uwc.drop(['atom_type'],axis=1)

#ONLY BACKBONE ATOMS
atoms = ['C','CA','N']
bb_6fat = df_6fat[df_6fat.atom_type.isin(atoms)]
bb_6fat = bb_6fat.drop(['atom_type'],axis=1)

bb_1uwc = df_1uwc[df_1uwc.atom_type.isin(atoms)]
bb_1uwc = bb_1uwc.drop(['atom_type'],axis=1)

#Construct the minimum c-RMSD function
def c_RMSD(x,y):
    xc = np.mean(x)
    yc = np.mean(y)

    #optimal translation - substracting centroid from each point of x and y respectively.
    X = x - xc
    Y = y -yc

    #optimal rotation
    Xtr = X.transpose()
    XtrY = np.dot(Xtr,Y)
    u, s, vtr = np.linalg.svd(XtrY)
    
    #estimate rotational matrix Q
    q = np.dot(u,vtr)
    det_q = np.linalg.det(q)
    if det_q < 0:
        q = np.dot((u[0], u[1], -u[2]),vtr)

    #calculate min RMSD
    final_matrix = np.dot(X,q) - Y
    RMSD = np.linalg.norm(final_matrix)/np.sqrt(len(final_matrix))
    return RMSD

print(f"The minimum c-RMSD distance between the all atoms of the identical amino acids of 1uwc and 6fat is {c_RMSD(common_1uwc,common_6fat)} Å.")
print(f"The minimum c-RMSD distance between the CA atoms of 1uwc and 6fat is {c_RMSD(CA_1uwc,CA_6fat)} Å.")
print(f"The minimum c-RMSD distance between the backbone atoms of 1uwc and 6fat is {c_RMSD(bb_1uwc,bb_6fat)} Å.")


The minimum c-RMSD distance between the all atoms of the identical amino acids of 1uwc and 6fat is 1.7704694008641126 Å.
The minimum c-RMSD distance between the CA atoms of 1uwc and 6fat is 2.271337595191768 Å.
The minimum c-RMSD distance between the backbone atoms of 1uwc and 6fat is 2.4684369249815084 Å.



C. Select 3 residues at the calcium binding site namely Val276-Ala277-Asp278 in the 3D structure of Feruloyl esterase with PDB code 6FAT. Consider only their 3 backbone atoms (smallest indices) namely atoms N, Ca, C of each, as points with 3D coordinates. Construct the corresponding Cayley-Menger (border) matrix B of dimension 10x10.

In [4]:

#Loading the coordinates of the 3 residues as pandas dataframe.
data = pd.read_csv("276_278_backbone.txt", sep = "\s+",header = None, names = ['x','y','z'], usecols=[6,7,8])

#Compute distance matrix M
M = (distance_matrix(data,data)**2)/2

#Add the 0th row and 0th column to the M-matrix
col0 = np.ones((len(M),1))
row0 = [0,1,1,1,1,1,1,1,1,1]
B = np.hstack((col0,M))
B = np.vstack((row0,B))


labels = ['0','p1','p2','p3','p4','p5','p6','p7','p8','p9']
#The resulting matrix is the border matrix B of size 10x10
print("=============================================\B matrix/===============================================")
pd.DataFrame(B,columns = labels, index = labels)

=============================================\B matrix/===============================================


,0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
p1,1.0,0.000000,1.077665,3.000898,5.474625,10.628379,15.588145,16.357075,23.977149,31.746657
p2,1.0,1.077665,0.000000,1.173865,2.949705,7.310758,10.191953,10.845074,16.593745,24.158012
p3,1.0,3.000898,1.173865,0.000000,0.890811,3.025239,5.042831,5.417763,10.345519,15.923701
p4,1.0,5.474625,2.949705,0.890811,0.000000,1.073002,3.038940,4.841754,9.960261,15.778275
p5,1.0,10.628379,7.310758,3.025239,1.073002,0.000000,1.169738,2.939012,7.337819,11.425268
p6,1.0,15.588145,10.191953,5.042831,3.038940,1.169738,0.000000,0.888330,3.055223,6.377039
p7,1.0,16.357075,10.845074,5.417763,4.841754,2.939012,0.888330,0.000000,1.084853,3.162624
p8,1.0,23.977149,16.593745,10.345519,9.960261,7.337819,3.055223,1.084853,0.000000,1.183364
p9,1.0,31.746657,24.158012,15.923701,15.778275,11.425268,6.377039,3.162624,1.183364,0.000000


D. Check that rank(B)=5. Compute the Gram matrix G, and its SVD so as to get 3D point coordinates. Check if it is the same structure, by computing its c-RMSD to (c).

In [5]:
#Calculate the rank of the border matrix (B)
rank_B = np.linalg.matrix_rank(B)
print(f"The rank of B-matrix is {rank_B}.\n")

#Define function that computes G matrix and the predicted values of points' coordinates
def predicted_P(M):
  #Compute G using distances of M matrix, but for each cell use the formula:
  #Gij = Mi1 - Mij + M1j, where p11 --> origin
  G = np.zeros((9,9))
  for i in range(len(G)):
    for j in range(len(G)):
      G[i][j] = M[1][j] - M[i][j] + M[i][1]
  #Calculate V,S,V-transpose matrices
  V, S, VT = np.linalg.svd(G)
  V = V[:,0:3] #keep only the first 3 columns of V matrix
  VT = VT[0:3] #keep only the first 3 rows of V-transpose matrix
  S = np.array([[S[0],0,0],[0,S[1],0],[0,0,S[2]]]) #reshape S matrix, so that it's a 3x3 matrix. Keep the first 3 elements of S.

  #Calculate the predicted values of points' coordinates
  output = pd.DataFrame(np.dot(V,np.sqrt(S)))
  return output, G

predicted_values, gram_matrix = predicted_P(M)

print(f"The minimum c-RMSD distance between the true and predicted values is {c_RMSD(data,predicted_values)} Å.\n")
print("=====================================\Gram matrix/===========================================")
pd.DataFrame(gram_matrix,columns= labels[1:], index = labels[1:])

The rank of B-matrix is 5.

The minimum c-RMSD distance between the true and predicted values is 8.303908659103458e-15 Å.

=====================================\Gram matrix/===========================================


,p1,p2,p3,p4,p5,p6,p7,p8,p9
p1,2.155330,0.0,-0.749368,-1.447254,-2.239956,-4.318526,-4.434336,-6.305739,-6.510980
p2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
p3,-0.749368,0.0,2.347730,3.232760,5.459384,6.322988,6.601176,7.422091,9.408176
p4,-1.447254,0.0,3.232760,5.899411,9.187462,10.102719,8.953026,9.583190,11.329443
p5,-2.239956,0.0,5.459384,9.187462,14.621516,16.332974,15.216820,16.566684,20.043502
p6,-4.318526,0.0,6.322988,10.102719,16.332974,20.383907,20.148698,23.730476,27.972927
p7,-4.434336,0.0,6.601176,8.953026,15.216820,20.148698,21.690148,26.353966,31.840462
p8,-6.305739,0.0,7.422091,9.583190,16.566684,23.730476,26.353966,33.187490,39.568393
p9,-6.510980,0.0,9.408176,11.329443,20.043502,27.972927,31.840462,39.568393,48.316025


E. Perturb entries of B by a percentage, then compute G, apply SVD so G=UΣUT. Let S be the diagonal matrix containing zeros and the 3 largest 
singular values of G. Suppose G ~ USUT, compute the 3D coordinates and report the c-RMSD against the actual structure. What is the max perturbation that yields a structure whose c-RMSD <R =1 Angstrom (or another meaningful bound R) with the real structure?

In [6]:
n = 1
RMSD = 0.0
outcome = []
#Perturb the initial set of coordinates, until the resolnving RMSD >= 1. 
#Start by 0% of perturbation and increase by 0.5% in each iteration.
while RMSD < 1:
  perturbed_data,g = predicted_P(M*n)
  perturbed_data = pd.DataFrame(perturbed_data)
  n = n - 0.005
  RMSD = c_RMSD(data,perturbed_data)
  outcome.append([round(1-n,3)*100,RMSD])

outcome = pd.DataFrame(outcome, columns = ['perturbation(%)','c-RMSD(Å)'])
print(f"The max perturbation, that yields a structure whose c-RMSD <R = 1 Å with the real structure, is {outcome.iloc[len(outcome)-2,0]}%.\n")
outcome


The max perturbation, that yields a structure whose c-RMSD <R = 1 Å with the real structure, is 60.5%.



,perturbation(%),c-RMSD(Å)
0,0.5,8.303909e-15
1,1.0,6.756055e-03
2,1.5,1.352911e-02
3,2.0,2.031928e-02
4,2.5,2.712672e-02
...,...,...
117,59.0,9.603051e-01
118,59.5,9.708111e-01
119,60.0,9.813814e-01
120,60.5,9.920172e-01


F. Repeat (e) but instead of perturbing, keep only distances of B which are less than some cutoff T and estimate T such that the computed structure has c-RMSD <R against the input structure. Show the values of T you tried and the resulting c-RMSD.

In [7]:
def cutoff_P(M, T):
  #compute G using distances of the orginal distance matrix,
  #but make the distances that are reater than the threshold, equal to the threshold.
  m = M.copy()
  m[m > T] = T
  G = np.zeros((9,9))
  for i in range(len(G)):
    for j in range(len(G)):
      G[i][j] = m[1][j] - m[i][j] + m[i][1]
  G = pd.DataFrame(G)

  #Calculate V,S,V-transpose matrices
  V, S, VT = np.linalg.svd(G)
  V = V[:,0:3] #keep only the first 3 columns of V matrix
  VT = VT[0:3] #keep only the first 3 rows of V-transpose matrix
  S = np.array([[S[0],0,0],[0,S[1],0],[0,0,S[2]]]) #reshape S matrix, so that it's a 3x3 matrix. Keep the first 3 elements of S.

  #Calculate the predicted values of points' coordinates
  output = pd.DataFrame(np.dot(V,np.sqrt(S)))
  return output

T_matrix = []
for t in range(3, 33):
  pred_coors = cutoff_P(M, t)
  RMSD = c_RMSD(data, pred_coors)
  T_matrix.append([t,RMSD])

pd.DataFrame(T_matrix, columns = ['Cutoff T(Å)', 'c_RMSD(Å)'])


,Cutoff T(Å),c_RMSD(Å)
0,3,2.014306e+00
1,4,1.858290e+00
2,5,1.747092e+00
3,6,1.650325e+00
4,7,1.556571e+00
5,8,1.445790e+00
6,9,1.358249e+00
7,10,1.315567e+00
8,11,1.307499e+00
9,12,1.204054e+00
